# Preprocessing

- Cleaning parallel corpus
- BPE tokenization

[Bicleaner](https://github.com/bitextor/bicleaner-hardrules)


RULES:
- no_empty,	Sentence is empty
- not_too_long,	Sentence is more than 1024 characters long
- not_too_short,	Sentence is less than	3 words long
- length_ratio,	The length ratio between the source sentence and target sentence (in bytes) is too low or too high
- no_identical,	Alphabetic content in source sentence and target sentence is identical
- no_literals,  Unwanted literals: "Re:","{{", "%s", "}}", "+++", "***", '=\"'
- no_only_symbols,	The ratio of non-alphabetic characters in source sentence is more than 90%
- no_only_numbers,	The ratio of numeric characters in source sentence is too high
- no_urls,	There are URLs (disabled by default)
- no_breadcrumbs,	There are more than 2 breadcrumb characters in the sentence
- no_glued_words,	There are words in the sentence containing too many uppercased characters between lowercased characters
- no_repeated_words, There are words repeated consecutively
- no_unicode_noise,	Too many characters from unwanted unicode in source sentence
- no_space_noise,	Too many consecutive single characters separated by spaces in the sentence (excludes digits)
- no_paren,	Too many parenthesis or brackets in sentence
- no_escaped_unicode,	There is unescaped unicode characters in sentence
- no_bad_encoding,	Source sentence or target sentence contains mojibake
- no_titles,	All words in source sentence or target sentence are uppercased or in titlecase
- no_wrong_language,	Sentence is not in the desired language
- no_porn,	Source sentence or target sentence contains text identified as porn
- no_number_inconsistencies,	Sentence contains different numbers in source and target (disabled by default)
- no_script,_inconsistencies	Sentence source or target contains characters from different script/writing systems (disabled by default)
- lm_filter,	The sentence pair has low fluency score from the language model

In [1]:
#install dependency libraries
!apt install libhunspell-dev
!apt-get install hunspell-en-us
#hunspell-en-med ??
!apt-get install hunspell-de-de
!pip install hunspell

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  dictionaries-common hunspell-en-us libhunspell-1.7-0 libtext-iconv-perl
Suggested packages:
  ispell | aspell | hunspell wordlist hunspell openoffice.org-hunspell | openoffice.org-core
The following NEW packages will be installed:
  dictionaries-common hunspell-en-us libhunspell-1.7-0 libhunspell-dev libtext-iconv-perl
0 upgraded, 5 newly installed, 0 to remove and 29 not upgraded.
Need to get 896 kB of archives.
After this operation, 3,130 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libtext-iconv-perl amd64 1.7-7build3 [14.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 dictionaries-common all 1.28.14 [185 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 hunspell-en-us all 1:2020.12.07-2 [280 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libhun

In [2]:
#install bicleaner and hard-rules 2.11
!pip install --config-settings="--build-option=--max_order=7" https://github.com/kpu/kenlm/archive/master.zip

     \ 553.6 kB 10.4 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp311-cp311-linux_x86_64.whl size=3186999 sha256=669415c8ed987fcf7a6165ae9f7c82dd98d788949de17a62332acdcb06f86345
  Stored in directory: /tmp/pip-ephem-wheel-cache-o104rij6/wheels/4e/ca/6a/e5da175b1396483f6f410cdb4cfe8bc8fa5e12088e91d60413
Successfully built kenlm


In [3]:
!pip list > requirements.txt
!cat requirements.txt       #to show versions of all libraries

Package                            Version
---------------------------------- -------------------
absl-py                            1.4.0
accelerate                         1.3.0
aiohappyeyeballs                   2.6.1
aiohttp                            3.11.13
aiosignal                          1.3.2
alabaster                          1.0.0
albucore                           0.0.23
albumentations                     2.0.5
ale-py                             0.10.2
altair                             5.5.0
annotated-types                    0.7.0
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.7.1
arviz                              0.20.0
astropy                            7.0.1
astropy-iers-data                  0.2025.3.10.0.29.26
astunparse                         1.6.3
atpublic                           4.1.0
attrs                              25.3.0
audioread          

In [4]:
#clone repo from Github and navigate to correct working directory
!git clone https://github.com/fubotz/BMT_2025S

Cloning into 'BMT_2025S'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 61 (delta 18), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (61/61), 5.46 MiB | 4.78 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [1]:
%cd BMT_2025S/week2_files/Basic-MT_week2_files      #again after runningtime reset

/content/BMT_2025S/week2_files/Basic-MT_week2_files


In [6]:
#load parallel corpus
#check number of lines
!wc -l dev*

   500 dev.en-de.de
   500 dev.en-de.en
  1000 total


In [7]:
#bicleanaer requires parallel data into the same file with columns en-de
!paste dev.en-de.en dev.en-de.de > dev.en-de

In [8]:
#check output
!head dev.en-de

Yevonde's most famous work was inspired by a theme party held on 5 March 1935, where guests dressed as Roman and Greek gods and goddesses.	Besonders bekannt wurden ihre Aufnahmen von einem Fest 1935, zu dem Gäste als griechische Götter und Göttinnen verkleidet kamen.
Mora is working on a trilogy about the IT specialist Darius Kopp, of which band I "The Only Man on the Continent" and Volume II "The Monster" have already appeared.	Terézia Mora arbeitet an einer Trilogie um den IT-Spezialisten Darius Kopp, von der Band I „Der einzige Mann auf dem Kontinent“ und Band II „Das Ungeheuer“ bereits erschienen sind.
The first person to enter this section was Günther J. Wolf with seven members of his ice course.	Eine erste Befahrung dieses Abschnitts gelang Günther J. Wolf mit sieben Teilnehmern seines Eiskurses.
They were renumbered in 1970 to 100 903 and 904, and in 1973 to 199 003 and 004.	Sie wurden 1970 in 100 903 und 904, 1973 in 199 003 und 004 umgenummert.
The grave is probably a disturbe

In [9]:
!pip install cyhunspell

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cyhunspell
  Running setup.py clean for cyhunspell
Failed to build cyhunspell
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (cyhunspell)


In [2]:
!pip install numpy==1.24

In [3]:
!pip install bicleaner-hardrules

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of fastspell to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of fastspell to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 83.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 

In [4]:
#apply bicleaner
!bicleaner-hardrules  \
        -s en -t de \
        dev.en-de  \
        dev.en-de.classified

2025-03-18 13:20:47,989 - WARNING - Downloading FastText model...
2025-03-18 13:20:49,665 - WARNING - Metadata file not provided.
2025-03-18 13:20:49,665 - INFO - LM filtering disabled.
2025-03-18 13:20:49,665 - INFO - Porn removal disabled.
2025-03-18 13:20:49,694 - INFO - Executing main program...
2025-03-18 13:20:49,694 - INFO - Starting process
2025-03-18 13:20:49,694 - INFO - Running 1 workers at 10000 rows per block
2025-03-18 13:20:49,718 - INFO - Start mapping
2025-03-18 13:20:49,736 - INFO - End mapping
2025-03-18 13:20:53,089 - INFO - Hard rules applied. Output available in dev.en-de.classified
2025-03-18 13:20:53,094 - INFO - Finished
2025-03-18 13:20:53,094 - INFO - Total: 500 rows
2025-03-18 13:20:53,094 - INFO - Elapsed time 3.40 s
2025-03-18 13:20:53,094 - INFO - Troughput: 147 rows/s
2025-03-18 13:20:53,095 - INFO - Program finished


In [5]:
#check file
!head dev.en-de.classified

Yevonde's most famous work was inspired by a theme party held on 5 March 1935, where guests dressed as Roman and Greek gods and goddesses.	Besonders bekannt wurden ihre Aufnahmen von einem Fest 1935, zu dem Gäste als griechische Götter und Göttinnen verkleidet kamen.	1
Mora is working on a trilogy about the IT specialist Darius Kopp, of which band I "The Only Man on the Continent" and Volume II "The Monster" have already appeared.	Terézia Mora arbeitet an einer Trilogie um den IT-Spezialisten Darius Kopp, von der Band I „Der einzige Mann auf dem Kontinent“ und Band II „Das Ungeheuer“ bereits erschienen sind.	1
The first person to enter this section was Günther J. Wolf with seven members of his ice course.	Eine erste Befahrung dieses Abschnitts gelang Günther J. Wolf mit sieben Teilnehmern seines Eiskurses.	1
They were renumbered in 1970 to 100 903 and 904, and in 1973 to 199 003 and 004.	Sie wurden 1970 in 100 903 und 904, 1973 in 199 003 und 004 umgenummert.	1
The grave is probably a 

In [6]:
#select only 1
!grep '1$' dev.en-de.classified >  dev.en-de.clean

In [7]:
!grep '0$' dev.en-de.classified >  dev.en-de.filter

In [8]:
#check files
!wc -l dev.en-de.classified
!wc -l dev.en-de.clean

500 dev.en-de.classified
466 dev.en-de.clean


In [9]:
!head -n 50 dev.en-de.clean

Yevonde's most famous work was inspired by a theme party held on 5 March 1935, where guests dressed as Roman and Greek gods and goddesses.	Besonders bekannt wurden ihre Aufnahmen von einem Fest 1935, zu dem Gäste als griechische Götter und Göttinnen verkleidet kamen.	1
Mora is working on a trilogy about the IT specialist Darius Kopp, of which band I "The Only Man on the Continent" and Volume II "The Monster" have already appeared.	Terézia Mora arbeitet an einer Trilogie um den IT-Spezialisten Darius Kopp, von der Band I „Der einzige Mann auf dem Kontinent“ und Band II „Das Ungeheuer“ bereits erschienen sind.	1
The first person to enter this section was Günther J. Wolf with seven members of his ice course.	Eine erste Befahrung dieses Abschnitts gelang Günther J. Wolf mit sieben Teilnehmern seines Eiskurses.	1
They were renumbered in 1970 to 100 903 and 904, and in 1973 to 199 003 and 004.	Sie wurden 1970 in 100 903 und 904, 1973 in 199 003 und 004 umgenummert.	1
The grave is probably a 

In [10]:
!head -n 50 dev.en-de.filter

He was an editor of the journals: Zeitschrift für Tropenmedizin, the Zentralblatt für Bakteriologie and the Zeitschrift für Parasitenkunde.	Ferner war er Herausgeber der Zeitschrift für Tropenmedizin, dem Zentralblatt für Bakteriologie und der Zeitschrift für Parasitenkunde.	0
"Das Himmelreich zu Erlangen – offen aus Tradition?"	Das Himmelreich zu Erlangen – offen aus Tradition?	0
"Wörterbuch zur Sprache und Kultur der Twareg".	Prasse: Wörterbuch zur Sprache und Kultur der Twareg.	0
Sensors and Actuators B: Chemical.	In: Sensors and Actuators B: Chemical.	0
The Daily Courier.	In: The Daily Courier.	0
Competitivitat de l´economia catalana en l´horitzó 2010: Effectes macroeconòmics del dèfiit fiscal amb l´Estat espanyol (Competitivity of the Catalan economy in the horizon 2010: Macroeconomic effects of the fiscal deficit with the Spanish State) - 2003 Polítiques públiques: Una visió renovada (Public politics: An updated perspective) - 2004 L´espoli fiscal.	Competitivitat de l´economia ca

In [11]:
#split file into columns
!cut -f1 dev.en-de.clean > dev.en-de.clean.en
!cut -f2 dev.en-de.clean > dev.en-de.clean.de

In [12]:
#check files
!wc -l dev.en-de.clean.en
!wc -l dev.en-de.clean.de

466 dev.en-de.clean.en
466 dev.en-de.clean.de


# TODO BICLEAN
  - Training data 50k, dev 500, and test 500
  - clean it with hard rules


*paper: https://aclanthology.org/2020.eamt-1.31.pdf



In [14]:
#combine english and german sentences for each set
#delete existing dev* files first or overwrite
!paste train.en-de.en train.en-de.de > train.en-de
!paste dev.en-de.en dev.en-de.de > dev.en-de
!paste test.en-de.en test.en-de.de > test.en-de

In [19]:
!wc -l train.en-de dev.en-de test.en-de

   50000 train.en-de
     500 dev.en-de
     500 test.en-de
   51000 total


In [15]:
#apply bicleaner to each dataset
!bicleaner-hardrules -s en -t de train.en-de train.en-de.classified
!bicleaner-hardrules -s en -t de dev.en-de dev.en-de.classified
!bicleaner-hardrules -s en -t de test.en-de test.en-de.classified

2025-03-18 13:48:44,757 - WARNING - Metadata file not provided.
2025-03-18 13:48:44,757 - INFO - LM filtering disabled.
2025-03-18 13:48:44,757 - INFO - Porn removal disabled.
2025-03-18 13:48:44,776 - INFO - Executing main program...
2025-03-18 13:48:44,776 - INFO - Starting process
2025-03-18 13:48:44,776 - INFO - Running 1 workers at 10000 rows per block
2025-03-18 13:48:44,788 - INFO - Start mapping
2025-03-18 13:48:44,972 - INFO - End mapping
2025-03-18 13:50:06,930 - INFO - Hard rules applied. Output available in train.en-de.classified
2025-03-18 13:50:06,935 - INFO - Finished
2025-03-18 13:50:06,936 - INFO - Total: 50000 rows
2025-03-18 13:50:06,936 - INFO - Elapsed time 82.16 s
2025-03-18 13:50:06,936 - INFO - Troughput: 608 rows/s
2025-03-18 13:50:06,937 - INFO - Program finished
2025-03-18 13:50:08,543 - WARNING - Metadata file not provided.
2025-03-18 13:50:08,543 - INFO - LM filtering disabled.
2025-03-18 13:50:08,543 - INFO - Porn removal disabled.
2025-03-18 13:50:08,559 

In [16]:
#extract only clean sentences (1)
!grep '1$' train.en-de.classified > train.en-de.clean
!grep '1$' dev.en-de.classified > dev.en-de.clean
!grep '1$' test.en-de.classified > test.en-de.clean

In [17]:
#split back into english and german files
!cut -f1 train.en-de.clean > train.en-de.clean.en
!cut -f2 train.en-de.clean > train.en-de.clean.de

!cut -f1 dev.en-de.clean > dev.en-de.clean.en
!cut -f2 dev.en-de.clean > dev.en-de.clean.de

!cut -f1 test.en-de.clean > test.en-de.clean.en
!cut -f2 test.en-de.clean > test.en-de.clean.de

In [18]:
#check final datasets
!wc -l train.en-de.clean train.en-de.clean.en train.en-de.clean.de
!wc -l dev.en-de.clean dev.en-de.clean.en dev.en-de.clean.de
!wc -l test.en-de.clean test.en-de.clean.en test.en-de.clean.de

   46612 train.en-de.clean
   46612 train.en-de.clean.en
   46612 train.en-de.clean.de
  139836 total
   466 dev.en-de.clean
   466 dev.en-de.clean.en
   466 dev.en-de.clean.de
  1398 total
   467 test.en-de.clean
   467 test.en-de.clean.en
   467 test.en-de.clean.de
  1401 total


# BPE

from [subword-nmt](https://github.com/rsennrich/subword-nmt)

In [20]:
#install subword nmt
!pip install subword-nmt #==0.3.8

In [21]:
#learn bpe model
!subword-nmt learn-joint-bpe-and-vocab --input train.en-de.clean.en train.en-de.clean.de -s 16000 -o train.bpe --write-vocabulary train.vocab.en train.vocab.de

100% 16000/16000 [01:47<00:00, 148.88it/s]


In [22]:
#apply bpe source (NB: +added vocabulary control)
#original: !subword-nmt apply-bpe -c train.bpe < dev.en-de.en > dev.en-de.bpe.en

# Training set
!subword-nmt apply-bpe -c train.bpe --vocabulary train.vocab.en --vocabulary-threshold 50 < train.en-de.clean.en > train.en-de.bpe.en
!subword-nmt apply-bpe -c train.bpe --vocabulary train.vocab.de --vocabulary-threshold 50 < train.en-de.clean.de > train.en-de.bpe.de

# Dev set
!subword-nmt apply-bpe -c train.bpe --vocabulary train.vocab.en --vocabulary-threshold 50 < dev.en-de.clean.en > dev.en-de.bpe.en
!subword-nmt apply-bpe -c train.bpe --vocabulary train.vocab.de --vocabulary-threshold 50 < dev.en-de.clean.de > dev.en-de.bpe.de

# Test set
!subword-nmt apply-bpe -c train.bpe --vocabulary train.vocab.en --vocabulary-threshold 50 < test.en-de.clean.en > test.en-de.bpe.en
!subword-nmt apply-bpe -c train.bpe --vocabulary train.vocab.de --vocabulary-threshold 50 < test.en-de.clean.de > test.en-de.bpe.de

In [24]:
#check out bpe train
!head train.en-de.bpe.en

A recent analysis by Ap@@ al@@ de@@ tt@@ i et al. (201@@ 1) suggest@@ s that G@@ ong@@ x@@ ian@@ os@@ aur@@ us was more bas@@ al than V@@ ul@@ can@@ od@@ on, T@@ az@@ ou@@ d@@ as@@ aur@@ us and Is@@ an@@ os@@ aur@@ us, but more der@@ i@@ ved than the early s@@ aur@@ op@@ o@@ ds An@@ t@@ et@@ on@@ it@@ r@@ us, L@@ ess@@ em@@ s@@ aur@@ us, B@@ lik@@ an@@ as@@ aur@@ us, Cam@@ el@@ o@@ ti@@ a and Mel@@ an@@ or@@ os@@ aur@@ us.
R@@ ei@@ ch@@ h@@ art also carried out execu@@ tions in C@@ olog@@ ne, Fran@@ k@@ fur@@ t-@@ Pre@@ ung@@ es@@ hei@@ m, Berlin@@ -@@ Pl@@ ö@@ tz@@ en@@ se@@ e, B@@ ran@@ den@@ burg@@ -@@ G@@ ör@@ den and B@@ res@@ lau@@ , where central execu@@ tion sit@@ es had also been construc@@ ted.
U@@ ph@@ old the right of all, without disc@@ ri@@ min@@ ation, to a natural and social en@@ vir@@ on@@ ment suppor@@ tive of human di@@ gn@@ ity, bo@@ di@@ ly health and spir@@ itu@@ al w@@ ell@@ -@@ b@@ ein@@ g, with special attention to the rights of in@@ di@@ gen@@ ous pe@@ op@@ le

In [25]:
#check out bpe train
!head dev.en-de.bpe.en

Y@@ ev@@ on@@ de@@ 's most famous work was inspired by a theme party held on 5 March 193@@ 5, where gu@@ ests d@@ ressed as Roman and Greek go@@ ds and go@@ d@@ de@@ ss@@ es.
Mor@@ a is working on a tri@@ log@@ y about the I@@ T speci@@ alist D@@ ari@@ us K@@ op@@ p@@ , of which band I "The Only M@@ an on the Con@@ tin@@ ent@@ " and Vol@@ um@@ e II "The Mon@@ ster@@ " have already appear@@ ed.
The first person to ent@@ er this section was G@@ ü@@ n@@ ther J. Wol@@ f with seven members of his ice cour@@ se.
They were ren@@ um@@ ber@@ ed in 1970 to 100 9@@ 0@@ 3 and 9@@ 0@@ 4, and in 197@@ 3 to 19@@ 9 00@@ 3 and 00@@ 4.
The gra@@ ve is probably a dist@@ ur@@ bed ar@@ ran@@ ge@@ ment, which was covered earlier with wood or ston@@ es.
Per@@ sec@@ u@@ tions ended following John@@ 's death on 23 May 16@@ 7@@ 7, at the age of 7@@ 4.
In celeb@@ ration he wrote a book entitled Three Vis@@ its to Mad@@ ag@@ as@@ car (@@ 185@@ 8@@ ).
Berlin@@ ale Tal@@ ents and Per@@ sp@@ ek@@ tive De@@ ut@@ sch@

In [26]:
#check out bpe train
!head test.en-de.bpe.en

The EC@@ F used to p@@ ub@@ l@@ ish a new@@ s@@ letter Ch@@ ess M@@ ov@@ es, which was free to member@@ s.
Cape Bar@@ r@@ en, with the other is@@ lands in the Fur@@ ne@@ au@@ x Gro@@ up@@ , are a popular dest@@ ination for sea k@@ ay@@ ak@@ ers who attempt the crossing of B@@ ass St@@ ra@@ it from the Australian main@@ land at Wil@@ sons Pro@@ mon@@ tor@@ y, Vic@@ tori@@ a to the T@@ as@@ man@@ ian main@@ land.
The work is ar@@ tic@@ ulated in a single mov@@ ement, and comp@@ ris@@ es an ens@@ em@@ ble con@@ si@@ st@@ ing of two viol@@ in@@ s, a cel@@ lo@@ , a pian@@ o, a flu@@ te, and a pic@@ col@@ o, which was recorded in Russi@@ a by sol@@ o@@ ists of the Mos@@ co@@ w Ph@@ il@@ har@@ mon@@ ic Or@@ ch@@ est@@ ra bar@@ ely two days after its open@@ ing.
The base is reported to h@@ ost several M@@ Q@@ -@@ 9 Re@@ ap@@ er d@@ ron@@ es, based on s@@ at@@ ell@@ ite im@@ ag@@ er@@ y.
Some 80@@ ,000 people are ho@@ sp@@ it@@ alized there every year, and another 6@@ 00@@ ,000 are tre@@ ated i

# TODO BPE (see above)

- Train bpe model with the training data
- Apply on training, dev, and test

**NOTE:** to get original segmentation use


```
!sed -r 's/(@@ )|(@@ ?$)//g' < file_in > file_out
```



In [27]:
!sed -r 's/(@@ )|(@@ ?$)//g' < train.en-de.bpe.en > train.en-de.clean_bpe.en
!sed -r 's/(@@ )|(@@ ?$)//g' < train.en-de.bpe.de > train.en-de.clean_bpe.de

!sed -r 's/(@@ )|(@@ ?$)//g' < dev.en-de.bpe.en > dev.en-de.clean_bpe.en
!sed -r 's/(@@ )|(@@ ?$)//g' < dev.en-de.bpe.de > dev.en-de.clean_bpe.de

!sed -r 's/(@@ )|(@@ ?$)//g' < test.en-de.bpe.en > test.en-de.clean_bpe.en
!sed -r 's/(@@ )|(@@ ?$)//g' < test.en-de.bpe.de > test.en-de.clean_bpe.de

In [31]:
#check out first 10 clean sentences without bpe from train set
!head train.en-de.clean_bpe.en

A recent analysis by Apaldetti et al. (2011) suggests that Gongxianosaurus was more basal than Vulcanodon, Tazoudasaurus and Isanosaurus, but more derived than the early sauropods Antetonitrus, Lessemsaurus, Blikanasaurus, Camelotia and Melanorosaurus.
Reichhart also carried out executions in Cologne, Frankfurt-Preungesheim, Berlin-Plötzensee, Brandenburg-Görden and Breslau, where central execution sites had also been constructed.
Uphold the right of all, without discrimination, to a natural and social environment supportive of human dignity, bodily health and spiritual well-being, with special attention to the rights of indigenous peoples and minorities.
Schinkel was not mentioned, however, in a document published in 1828 on the "Construction Designs of the Prussian State", in which town planner, August Adolph Günther, cites the lighthouse as a "design by the Royal Main Construction Agency."
In the same zoo, of which Lord Black was one of the directors, the monkey Bimbo had escaped on

In [32]:
!head train.en-de.clean_bpe.de

Eine neuere Analyse von Cecilia Apaldetti und Kollegen (2011) lässt darauf schließen, dass Gongxianosaurus basaler (ursprünglicher) war als Vulcanodon, Tazoudasaurus und Isanosaurus, aber stärker abgeleitet (fortgeschrittener) als die frühen Sauropoden Antetonitrus, Lessemsaurus, Blikanasaurus, Camelotia und Melanorosaurus.
Reichhart vollzog vertretungsweise auch Hinrichtungen in Köln, Frankfurt-Preungesheim, Berlin-Plötzensee, Brandenburg-Görden und Breslau, wo ebenfalls zentrale Hinrichtungsstätten eingerichtet worden waren.
Am Recht aller – ohne Ausnahme – auf eine natürliche und soziale Umwelt festhalten, welche Menschenwürde, körperliche Gesundheit und spirituelles Wohlergehen unterstützt.
Schinkel wurde jedoch in einem 1828 erschienenen Druckwerk über die „Bauausführungen des Preußischen Staates“ nicht erwähnt, in welchem der Oberbaurat August Adolph Günther den Leuchtturm einen „Entwurf der Königl.
Im selben Zoo, dessen Ko-Direktor Lord Black war, war Bimbo ausgebrochen und erst

last step: download files

- train.en-de.bpe.en
- train.en-de.bpe.de
- dev.en-de.bpe.en
- dev.en-de.bpe.de
- test.en-de.bpe.en
- test.en-de.bpe.de